# Tutorial

## Preliminaries

The following set of examples shows the user how to train a Masked Autoregressive Flow (MAF) and an example Kernel Density Estimator (KDE). We further demonstrate how to use `margarine` to estimate the Kullback Leibler divergence and Bayesian Dimensionality with the trained MAF and KDE.

We also demonstrate how to use the clustering feature.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In order to demonstrate the applications of the code we need to generate some mock samples and we can visualise the posterior distributions with `anesthetic`.

In [ ]:
from anesthetic import MCMCSamples

x = np.random.normal(0, 1, 1000)
y = np.random.normal(2, 0.5, 1000)

data = np.vstack([x, y]).T
weights = np.ones(len(data))

samples = MCMCSamples(data=data, weights=weights)

To visualise the posterior we use anesthetic.

In [ ]:
names = [i for i in range(2)]
samples.plot_2d(names)

## Masked Autoregressive Flows

Firstly we will look at training a Masked Autoregressive Flow or MAF with `margarine`. To train the MAF we first need to initalise the class with the samples and corresponding weights.

In [ ]:
import os
os.chdir('../')

from margarine.maf import MAF

bij = MAF(data, weights)
bij.train(10000, early_stop=True)

We can then generate uniformly weighted samples from the bijector using the following code which technically takes samples on the hypercube and transforms them into samples on the target posterior distribution,

In [ ]:
x = bij(np.random.uniform(0, 1, size=(len(data), data.shape[-1])))

maf_samples = MCMCSamples(data=x, weights=np.ones(len(x)))
axes = samples.plot_2d(names, alpha=0.5, label='Original')
maf_samples.plot_2d(axes, alpha=0.5, label='MAF')
axes.iloc[0, 0].legend()


Alternatively we can generate samples with the following code which takes in an integer and returns an array of shape (int, 5). The `.sample()` function is a proxy for `__call__`.

In [ ]:
x = bij.sample(5000)

We can then go ahead an calculate the corresponding kl divergence and Bayesian dimensionality. 

The samples presented here were generated using a gaussian likelihood and fitting with nested sampling for 5 parameters. We can use `anesthetic` to calculate the KL divergence and Bayesian dimensionality for the samples for comparison. We see very similar results and note that the similarity improves with the number of epochs.

In [ ]:
from margarine.marginal_stats import calculate

stats = calculate(bij).statistics()
print(stats.iloc[0, 0], samples.D())
print(stats.iloc[1, 0], samples.d())

# Kernel Density Estimators

We can perform a similar analysis using Kernel Density Estimators rather than MAFs which is done with the following code. Note that the generation of the 'trained' model is significantly quicker than when performed with the MAFs.

In [ ]:
from margarine.kde import KDE
kde = KDE(data, weights)
kde.generate_kde()
x = kde.sample(5000)

samples = MCMCSamples(data=x, weights=weights)
samples.plot_2d(names)

stats = calculate(kde).statistics()
print(stats.iloc[0, 0], '+(-)', stats.iloc[0, 2] - stats.iloc[0, 0], 
      '(', stats.iloc[0, 0] - stats.iloc[0, 1], ')', samples.D())
print(stats.iloc[1, 0], '+(-)', stats.iloc[1, 2] - stats.iloc[1, 0], 
      '(', stats.iloc[1, 0] - stats.iloc[1, 1], ')', samples.d())

Rather than using the `kde.sample()` function to generate samples we could transform samples from the hypercube with the following code and the `__call__()` function. However, we note that this is a much slower method of generating samples as it is designed to be bijective. Transformation from the hypercube is useful if we would like to use a trained KDE or MAF as the prior in a subseqeunt nested sampling run however is not necessary if we simply want to calcualte marginal Bayesian statistics.

In [ ]:
x = kde(np.random.uniform(0, 1, size=(10, data.shape[-1])))

# Clustering with margarine

The below example demonstrates how to perform clustering with `margarine` which can improve the accuracy of the emulation.

In [ ]:
x = np.concatenate([np.random.normal(0, 1, 500), np.random.normal(5, 0.3, 500)])
y = np.random.normal(0, 3, 1000)

data = np.vstack([x, y]).T

samples = MCMCSamples(data=data, weights=np.ones(len(data)))
samples.plot_2d(names)

In [ ]:
flow = MAF(data, np.ones(len(data)), clustering=True)
flow.train(10000, early_stop=True)

In [ ]:
flow_samples = flow.sample(5000)
flow_samples = MCMCSamples(data=flow_samples, weights=np.ones(len(flow_samples)))
flow_samples.plot_2d(names)